In [1]:
import dicom2nifti
import dicom2nifti.settings as settings
import matplotlib.pyplot as plt 
from glob import glob
import os
import torchio as tio

settings.disable_validate_slice_increment()

/ubc/ece/home/ra/other/manmeetp/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#converting dicom slices to nifti 3d volumes
#storing the DICOM files in the .nii.gz format for easier image processing
files_error = list()

dicom_input = "ordered_data/CT_Lymph_Nodes"
nifti_output = "3D_volumes_NIfTI_images"

folder = glob(dicom_input + '/*')
files_already_done = []
# print(os.listdir(nifti_output))
# print(folder)
i = 0
try:
    for patient in folder:
        name = patient.split("/")[2]
        file_name = str(name + ".nii.gz")
        if (file_name not in os.listdir(nifti_output)):
            dicom2nifti.dicom_series_to_nifti(patient, os.path.join(nifti_output, name + ".nii.gz"))
        else: 
            files_already_done.append(name)
        i += 1
except Exception as e:
    print(e)
    files_error.append(name)
    i += 1
finally:
    files_error.append(name)




The length of the pixel data in the dataset (261042 bytes) doesn't match the expected length (524288 bytes). The dataset may be corrupted or there may be an issue with the pixel data handler.


In [ ]:
from pydicom import dcmread
dcm_file_path = "ordered_data/CT_Lymph_Nodes/MED_LYMPH_087/09-14-2014-MEDLYMPH087-mediastinallymphnodes-32851/mediastinallymphnodes-64654/1-001.dcm"
ds = dcmread(dcm_file_path)
for element in ds:
    print(element)


In [ ]:
def load_scan(path):
    slices = [pydicom.dcmread(path + ‘/’ + s) for s in               
              os.listdir(path)]
    slices = [s for s in slices if 'SliceLocation' in s]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] —   
                          slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation — 
                      slices[1].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices

    
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [7]:
#now we apply isotropic resampling to the images to get a voxel spacing of 1mm in each direction

import nibabel as nib
import torchio as tio
import numpy as np

#THIS METHOD WILL OVERWRITE THE FILES IN DATA 
#the actual data
image_folder = "3D_volumes_NIfTI_images"
dest_input_folder = "training_set/resampled_ALL_data"

for image in os.listdir(image_folder):
    affine_matrix = nib.load(os.path.join(image_folder, image)).affine

    input = tio.ScalarImage(os.path.join(image_folder, image), to_mni=affine_matrix)
    transform = tio.Resample(1, pre_affine_name='to_mni')
    isotropic_image = transform(input)
    print(np.shape(isotropic_image))

    #according to the paper a resolution of 128 x 128 x 144 was chosen 
    #but we will resample it to 128 x 128 x depth so that we can choose 128 slices after
    padding = tio.Resample(target=(np.shape(isotropic_image)[2]/128, np.shape(isotropic_image)[2]/128, 1))
    padded_image = padding(isotropic_image)
    print(np.shape(padded_image))

    if (os.path.isfile(os.path.join(dest_input_folder,  image))):
        os.remove(os.path.join(dest_input_folder,  image))
        print("file removed")
    tio.Image.save(padded_image, path=os.path.join(dest_input_folder,  image))
    print("ressssaaaampled")


(1, 313, 313, 574)
(1, 128, 128, 574)
ressssaaaampled
(1, 450, 450, 746)
(1, 128, 128, 746)
ressssaaaampled
(1, 295, 295, 709)
(1, 128, 128, 709)
ressssaaaampled
(1, 347, 347, 783)
(1, 128, 128, 783)
ressssaaaampled
(1, 469, 469, 671)
(1, 128, 128, 671)
ressssaaaampled
(1, 450, 450, 696)
(1, 128, 128, 696)
ressssaaaampled
(1, 240, 240, 636)
(1, 128, 128, 636)
ressssaaaampled
(1, 489, 489, 696)
(1, 128, 128, 696)
ressssaaaampled
(1, 379, 379, 524)
(1, 128, 128, 524)
ressssaaaampled
(1, 254, 254, 666)
(1, 128, 128, 666)
ressssaaaampled
(1, 308, 308, 666)
(1, 128, 128, 666)
ressssaaaampled
(1, 287, 287, 661)
(1, 128, 128, 661)
ressssaaaampled
(1, 226, 226, 827)
(1, 128, 128, 827)
ressssaaaampled
(1, 254, 254, 771)
(1, 128, 128, 771)
ressssaaaampled
(1, 240, 240, 752)
(1, 128, 128, 752)
ressssaaaampled
(1, 313, 313, 816)
(1, 128, 128, 816)
ressssaaaampled
(1, 226, 226, 830)
(1, 128, 128, 830)
ressssaaaampled
(1, 283, 283, 715)
(1, 128, 128, 715)
ressssaaaampled
(1, 443, 443, 691)
(1, 128, 

In [11]:
print(len(os.listdir("ordered_data/CT_Lymph_Nodes")))

175


In [10]:
print(len(files_error))

1
